# Importing Required Libraries¶


In [1]:
!pip install datasets kagglehub
import kagglehub
import os

In [2]:
!pip install datasets
import torch
from transformers import (AutoTokenizer,
                          AutoModelForSeq2SeqLM,
                          Trainer ,
                          TrainingArguments
)
from datasets import load_dataset, DatasetDict, Dataset #Import Dataset class
import pandas as pd
import numpy as np
# Download latest version
path = kagglehub.dataset_download("patrickfleith/space-news-dataset")

print("Path to dataset files:", path)

# Get the list of files in the downloaded directory
files = os.listdir(path)
print(f"Files in directory: {files}")

# Assuming the dataset is in a csv file, load it using pandas, and then load into a datasets.Dataset object
if len(files) > 0:
    for file in files:
        if file.endswith(".csv"): # or some other relevant format
           df = pd.read_csv(os.path.join(path, file)) # modify this line to load the data correctly according to its format
           dataset = Dataset.from_pandas(df) #load the dataset from pandas dataframe.  Reference the Dataset class from the datasets module
           print(f"Dataset loaded from file: {file}")
           break
else:
    print("No files found in the downloaded dataset directory.")

Path to dataset files: /kaggle/input/space-news-dataset
Files in directory: ['spacenews.csv']
Dataset loaded from file: spacenews.csv


In [3]:
dataset

Dataset({
    features: ['title', 'url', 'content', 'author', 'date', 'postexcerpt'],
    num_rows: 20716
})

In [4]:
# prompt: convert the previuse into csv

import pandas as pd

# Assuming 'df' is the pandas DataFrame you created earlier
# Convert the DataFrame to a CSV file
csv_file_path = 'space_news.csv'  # Specify the desired file name
df.to_csv(csv_file_path, index=False)  # index=False prevents writing row indices to the CSV

# Now, you can download the CSV file using the following:
from google.colab import files
files.download(csv_file_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [5]:
df

,title,url,content,author,date,postexcerpt
0,Electron launches first NASA PREFIRE Earth sci...,https://spacenews.com/electron-launches-first-...,LOS ANGELES — Rocket Lab launched the first of...,Jeff Foust,"May 25, 2024",NaN
1,Space Development Agency to procure dedicated ...,https://spacenews.com/space-development-agency...,WASHINGTON — T he Space Development Agency (S...,Sandra Erwin,"May 25, 2024",NaN
2,Bringing generative artificial intelligence to...,https://spacenews.com/bringing-generative-arti...,"TAMPA, Fla. — Amazon Web Services is busy posi...",Jason Rainbow,"May 24, 2024",NaN
3,SpaceX sets early June launch of next Starship...,https://spacenews.com/spacex-sets-early-june-l...,LOS ANGELES — SpaceX has set a June 5 launch d...,Jeff Foust,"May 24, 2024",NaN
4,NASA and Boeing moving ahead with Starliner te...,https://spacenews.com/nasa-and-boeing-moving-a...,LOS ANGELES — NASA and Boeing are preparing fo...,Jeff Foust,"May 24, 2024",NaN
...,...,...,...,...,...,...
20711,Kendall lays out Pentagon thinking on future s...,https://spacenews.com/frank-kendall-at-wsbr/,"\nFrank Kendall, the Pentagon’s top acquisitio...",SpaceNews Staff,"February 25, 2016","Frank Kendall, the Pentagon’s top acquisition ..."
20712,A larger share of NOAA’s declining space budge...,https://spacenews.com/a-larger-share-of-noaas-...,Updated Feb. 10 at 10:18 p.m. Eastern The U.S....,Debra Werner,"February 10, 2016",The U.S. National Oceanic and Atmospheric Admi...
20713,Think Tank Turns Its Attention To Mars As 2016...,https://spacenews.com/think-tank-turns-its-att...,WASHINGTON — As NASA develops a long-term stra...,Jeff Foust,"June 11, 2015",As NASA develops a long-term strategy to suppo...
20714,House Bill Leaves Last Three JPSS Satellites i...,https://spacenews.com/no-money-for-noaa-weathe...,WASHINGTON — A spending bill the House passed ...,Dan Leone,"June 4, 2015",A spending bill the House passed June 3 would ...


In [6]:
df= df[["title","content"]]

In [7]:
df

,title,content
0,Electron launches first NASA PREFIRE Earth sci...,LOS ANGELES — Rocket Lab launched the first of...
1,Space Development Agency to procure dedicated ...,WASHINGTON — T he Space Development Agency (S...
2,Bringing generative artificial intelligence to...,"TAMPA, Fla. — Amazon Web Services is busy posi..."
3,SpaceX sets early June launch of next Starship...,LOS ANGELES — SpaceX has set a June 5 launch d...
4,NASA and Boeing moving ahead with Starliner te...,LOS ANGELES — NASA and Boeing are preparing fo...
...,...,...
20711,Kendall lays out Pentagon thinking on future s...,"\nFrank Kendall, the Pentagon’s top acquisitio..."
20712,A larger share of NOAA’s declining space budge...,Updated Feb. 10 at 10:18 p.m. Eastern The U.S....
20713,Think Tank Turns Its Attention To Mars As 2016...,WASHINGTON — As NASA develops a long-term stra...
20714,House Bill Leaves Last Three JPSS Satellites i...,WASHINGTON — A spending bill the House passed ...


In [8]:
df.duplicated().sum()

0

In [9]:
df.dropna(how='any',inplace=True)

<ipython-input-9-6340c630c5ac>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(how='any',inplace=True)


In [10]:
df["title"][0]

'Electron launches first NASA PREFIRE Earth science cubesat'

In [11]:
df["content"][0]

'LOS ANGELES — Rocket Lab launched the first of two NASA Earth science cubesats on an Electron rocket May 25, exactly seven years after the company’s first launch. The Electron lifted off from Rocket Lab’s Launch Complex 1 in New Zealand at 3:41 a.m. Eastern after a brief hold because of ground winds. It placed into a 525-kilometer sun-synchronous orbit a 6U cubesat that is part of a NASA mission called Polar Radiant Energy in the Far-InfraRed Experiment or PREFIRE. NASA announced it made contract with the PREFIRE cubesat five hours after liftoff, starting an in-orbit checkout of the satellite. It will be joined by a second, identical cubesat launched on another Electron. Both NASA and Rocket Lab said that launch will take place in the “coming days” but did not provide a specific date. NASA previously said it planned to launch the two cubesats no more than three weeks apart. The two PREFIRE cubesats, operating in different sun-synchronous orbital planes,  each carry a thermal infrared 

In [12]:
train_df = df.sample(frac =0.8,random_state=42)
test_df = df.drop(train_df.index)

In [13]:
# # Save the train and test sets as CSV files
# train_df.to_csv('/kaggle/working/train.csv', index=False)
# test_df.to_csv('/kaggle/working/test.csv', index=False)
# print("Train and test datasets saved as CSV.")

In [14]:
# # Load the train and test datasets from the saved CSV files
# dataset = load_dataset('csv', data_files={'train': '/kaggle/working/train.csv', 'test': '/kaggle/working/test.csv'})

In [15]:
# processed_dataset = Dataset.from_pandas(df,)

In [16]:
# processed_dataset

In [17]:
# processed_dataset.remove_columns(['__index_level_0__'])

In [18]:
df = Dataset.from_pandas(df)

In [19]:
df

Dataset({
    features: ['title', 'content', '__index_level_0__'],
    num_rows: 20551
})

In [20]:
# Create a DatasetDict from the Dataset objects
processed_dataset = DatasetDict({"train":Dataset.from_pandas(train_df), "test": Dataset.from_pandas(test_df)})

print(processed_dataset)

def preprocess_function(exampel):
  targets  = exampel["title"]
  input = exampel["content"]
  return {"input":input,"target":targets }



DatasetDict({
    train: Dataset({
        features: ['title', 'content', '__index_level_0__'],
        num_rows: 16441
    })
    test: Dataset({
        features: ['title', 'content', '__index_level_0__'],
        num_rows: 4110
    })
})


In [21]:
preprocess_data = processed_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/16441 [00:00<?, ? examples/s]

Map:   0%|          | 0/4110 [00:00<?, ? examples/s]

In [22]:
preprocess_data.remove_columns(['__index_level_0__'])

DatasetDict({
    train: Dataset({
        features: ['title', 'content', 'input', 'target'],
        num_rows: 16441
    })
    test: Dataset({
        features: ['title', 'content', 'input', 'target'],
        num_rows: 4110
    })
})

In [23]:
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [24]:
def tokenize_function(examples):
    # Tokenize the 'input_text' (article content) and 'target_text' (title) separately
    model_inputs = tokenizer(examples['input'], max_length=512, truncation=True, padding="max_length")

    # Tokenize target texts for titles
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['target'], max_length=128, truncation=True, padding="max_length")

    # Include the tokenized target as labels
    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

In [25]:
preprocess_data = preprocess_data.remove_columns(['title', 'content'])

In [26]:
preprocess_data

DatasetDict({
    train: Dataset({
        features: ['__index_level_0__', 'input', 'target'],
        num_rows: 16441
    })
    test: Dataset({
        features: ['__index_level_0__', 'input', 'target'],
        num_rows: 4110
    })
})

In [27]:
tokenized_datasets = preprocess_data.map(tokenize_function, batched=True)


Map:   0%|          | 0/16441 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3953: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/4110 [00:00<?, ? examples/s]

In [28]:
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [29]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
              (wo): 

In [30]:
tokenized_datasets["train"]["input_ids"][0]

[549,
 21337,
 2365,
 16270,
 3,
 318,
 2759,
 4160,
 18,
 1490,
 20,
 25322,
 17,
 628,
 6696,
 33,
 16,
 5129,
 13,
 8029,
 12469,
 3049,
 5,
 14405,
 46,
 605,
 2273,
 9409,
 19,
 72,
 2084,
 13,
 8,
 174,
 12,
 1349,
 95,
 731,
 4030,
 15607,
 5,
 312,
 32,
 18506,
 7,
 6,
 3,
 9,
 1826,
 349,
 24,
 10291,
 3,
 9,
 1229,
 13,
 1591,
 18,
 390,
 16092,
 7,
 24,
 1463,
 4820,
 16,
 15607,
 6,
 2162,
 3049,
 5,
 2307,
 24,
 34,
 141,
 4313,
 3,
 9,
 1055,
 13274,
 6,
 42,
 885,
 1295,
 6,
 344,
 8,
 86,
 89,
 11096,
 26,
 15001,
 3114,
 1950,
 24552,
 41,
 19426,
 134,
 61,
 11,
 8,
 15199,
 485,
 10771,
 4741,
 472,
 3587,
 1707,
 1881,
 4267,
 297,
 41,
 21320,
 4132,
 61,
 314,
 7605,
 16,
 301,
 13113,
 5,
 37,
 349,
 243,
 132,
 47,
 46,
 3241,
 8218,
 77,
 18,
 2915,
 1253,
 24,
 8,
 192,
 7605,
 7,
 133,
 8029,
 1599,
 44,
 431,
 10,
 3288,
 3,
 102,
 5,
 51,
 5,
 7345,
 3049,
 5,
 2838,
 46,
 491,
 6592,
 13,
 81,
 3,
 7015,
 20325,
 6,
 966,
 1461,
 756,
 8,
 690,
 13,
 16226

# Setting up the Training Arguments

In [31]:
training_args = TrainingArguments(
     output_dir='./results',              # Output directory
    evaluation_strategy="epoch",         # Evaluate at each epoch
    learning_rate=5e-5,                  # Learning rate
    per_device_train_batch_size=8,       # Training batch size
    per_device_eval_batch_size=8,        # Evaluation batch size
    num_train_epochs=3,                  # Number of epochs
    weight_decay=0.01,                   # Weight decay
    logging_dir='./logs',                # Directory for logging
    logging_steps=500,
    report_to = "none",
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


# Training the Model¶


In [32]:
trainer = Trainer(
    model=model,                         # The instantiated 🤗 Transformers model to be trained
    args=training_args,
    train_dataset=tokenized_datasets["train"],         # Training dataset
    eval_dataset=tokenized_datasets["test"],          # Evaluation dataset
    tokenizer=tokenizer
)

<ipython-input-32-d94d0e8b12f9>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [33]:
trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,0.315600,0.276238
2,0.299700,0.268889
3,0.290100,0.266480


TrainOutput(global_step=6168, training_loss=0.5883257992072792, metrics={'train_runtime': 1628.2445, 'train_samples_per_second': 30.292, 'train_steps_per_second': 3.788, 'total_flos': 9168677363515392.0, 'train_loss': 0.5883257992072792, 'epoch': 3.0})

In [34]:
eval_results = trainer.evaluate()

In [35]:
def generate_title(article_text):
    inputs = tokenizer(article_text,return_tensors="pt", padding=True, truncation=True).to(device)
    outputs = model.generate(inputs["input_ids"], max_length=128, num_beams=4, early_stopping=True)
    return tokenizer.decode(outputs[0],skip_special_tokens= True)



In [36]:
test_article = "NASA has announced plans for a new satellite launch aimed at studying the Earth's atmosphere."
print(f"Generated Title: {generate_title(test_article)}")

Generated Title: NASA plans new satellite launch to study atmosphere


In [37]:
from huggingface_hub import notebook_login

notebook_login()

In [38]:
trainer.push_to_hub("title-generator")

training_args.bin:   0%|          | 0.00/5.24k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Ah7med/results/commit/a08ab9c9969bbe5a975b39e68043981a784d1e9f', commit_message='title-generator', commit_description='', oid='a08ab9c9969bbe5a975b39e68043981a784d1e9f', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Ah7med/results', endpoint='https://huggingface.co', repo_type='model', repo_id='Ah7med/results'), pr_revision=None, pr_num=None)